In [122]:
#Imports
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.metrics import accuracy_score

In [123]:
#Load the testing and training data
df_test = pd.read_csv('data/Testing.csv')
df_train = pd.read_csv('Training.csv')

In [124]:
#Load the label encoder
le = preprocessing.LabelEncoder()

In [125]:
#Determine features and target of training and testing data
X_train = df_train.iloc[:,0:131]
x_test = df_test.iloc[:,0:131]
y_train_labelled = df_train.iloc[:,132]
Y_test_labelled = df_test.iloc[:,132]

In [126]:
#Encode labels
y_train = le.fit_transform(y_train_labelled)
Y_test = le.fit_transform(Y_test_labelled)

In [127]:
#Decision Tree Classifier
model_DT = DecisionTreeClassifier(max_leaf_nodes=42, class_weight='balanced')
model_DT.fit(X_train,y_train)
print("Accuracy Score of Decision tree Classifier : "+str(accuracy_score(Y_test, model_DT.predict(x_test))*100)+"%")

Accuracy Score of Decision tree Classifier : 97.61904761904762%


In [128]:
#Naive Bayes Classifier
model_NB = GaussianNB()
model_NB.fit(X_train,y_train)
print("Accuracy Score of Naive Bayes Classifier : "+str(accuracy_score(Y_test, model_NB.predict(x_test))*100)+"%")

Accuracy Score of Naive Bayes Classifier : 100.0%


In [129]:
#Support Vector Classifier
model_SVC = SVC(kernel='linear', C=1)
model_SVC.fit(X_train,y_train)
print("Accuracy Score of Support Vector Classifier : "+str(accuracy_score(Y_test, model_SVC.predict(x_test))*100)+"%")

Accuracy Score of Support Vector Classifier : 100.0%


In [130]:
#Softmax Classifier
model_SM = LogisticRegression(random_state=0, multi_class='multinomial')
model_SM.fit(X_train,y_train)
print("Accuracy Score of Softmax Classifier : "+str(accuracy_score(Y_test, model_SM.predict(x_test))*100)+"%")

Accuracy Score of Softmax Classifier : 97.61904761904762%
